In [6]:
# imports
import os
import csv
from Bb_rest_helper import Get_Config
from Bb_rest_helper import Auth_Helper
from Bb_rest_helper import Bb_Requests
from Bb_rest_helper import Bb_Utils

# set logging
utils = Bb_Utils()
utils.set_logging()

# set requests
reqs = Bb_Requests()

# REST Api Authentication
quick_auth_learn = utils.quick_auth('./credentials/learn_config.json', 'Learn')
quick_auth_collab = utils.quick_auth(
    './credentials/collab_config.json', 'Collaborate')

# Learn url and token
learn_token = quick_auth_learn['token']
learn_url = quick_auth_learn['url']

# Colalb url and token
collab_token = quick_auth_collab['token']
collab_url = quick_auth_collab['url']

print("Collab and Learn Token set!")

Collab and Learn Token set!


In [13]:
# Get course list
# This function has been updated to filter results by course avaiability. Pass 'Yes' for available courses and 'No' for unavailable,
# default value is 'No', so  by default this will return a list of unavailable courses

def get_course_list(toogle_course_status = 'No'):
    course_url = '/learn/api/public/v3/courses'
    params = {
        'fields': 'courseId,uuid,name,availability.available'
    }
    course_list = reqs.Bb_GET(learn_url, course_url, learn_token, params)
    filtered_course_list = []
    for c in course_list:
        if c['availability']['available'] == toogle_course_status:
            filtered_course_list.append(c)
    
    print(f'{len(filtered_course_list)} courses in the system with available set as: {toogle_course_status}')
    return filtered_course_list


courses = get_course_list('No')
utils.pretty_printer(courses)

147 courses in the system with available set as: No
[
    {
        "availability": {
            "available": "No"
        },
        "courseId": "04833398_copy",
        "name": "04833398_copy",
        "uuid": "d47a0d86c722471cb00dc674bf06ed95"
    },
    {
        "availability": {
            "available": "No"
        },
        "courseId": "04833398_new",
        "name": "04833398_new",
        "uuid": "feee62db4a5544a68c9ad834259c2eb8"
    },
    {
        "availability": {
            "available": "No"
        },
        "courseId": "100-100-100-00",
        "name": "Introduction to Algebra",
        "uuid": "07eff93cbe5f4684a525fe2f306723a7"
    },
    {
        "availability": {
            "available": "No"
        },
        "courseId": "100-100-100-01",
        "name": "Introduction to Algebra",
        "uuid": "2bd1a8dfbbfb49f4a7c629bd5daa4f8f"
    },
    {
        "availability": {
            "available": "No"
        },
        "courseId": "100-100-100-02",
        "na

In [12]:
# Get sessions by course/context
def get_sessions(course_list):
    with open('./reports/collab_report.csv', 'a', newline='') as csvfile:
        fields = ['Course id', 'Course name', 'Course available',
                  'Session id', 'Session name', 'Guests allowed']
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        writer.writeheader()
        for c in course_list:
            course_id = c['courseId']
            course_name = c['name']
            course_available = c['availability']['available']
            sessions_url = '/sessions'
            params = {
                'contextExtId': c['uuid'],
                'fields': 'results.id,results.name,results.allowGuest'
            }
            data = reqs.Bb_GET(collab_url, sessions_url, collab_token, params)
            for d in data:
                if data:
                    writer.writerow({'Course id': course_id, 'Course name': course_name,'Course available': course_available, 'Session id': d['id'],
                                     'Session name': d['name'], 'Guests allowed': d['allowGuest']})


session_list = get_sessions(courses)